<a href="https://colab.research.google.com/github/ValentinCord/HandsOnAI_2/blob/main/NLP_Transformer_evaluate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <span> NLP : Transformer prédiction du modèle </span>
<hr style="border-bottom: solid;background-color:light;color:black;">

<a id="section-1"></a>
# <span>1. Installation des packages</span>
<hr style="border-bottom: solid;background-color:light;color:black;">

In [1]:
!/opt/bin/nvidia-smi
!rm -rf sample_data

!pip3 install transformers
!pip3 install datasets
!pip install sentencepiece

/bin/bash: /opt/bin/nvidia-smi: No such file or directory
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 5.1 MB/s 
     |████████████████████████████████| 182 kB 46.3 MB/s 
     |████████████████████████████████| 7.6 MB 30.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 452 kB 5.2 MB/s 
     |████████████████████████████████| 212 kB 47.7 MB/s 
     |████████████████████████████████| 132 kB 47.4 MB/s 
     |████████████████████████████████| 127 kB 58.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 5.3 MB/s 


# <span>2. Imports </span>
<hr style="border-bottom: solid;background-color:light;color:black;">

In [2]:
# basics 
import os
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn import metrics

# transformers 
from datasets import load_dataset
from transformers import pipeline, AutoTokenizer
from transformers import CamembertModel, CamembertTokenizer
from transformers import DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

# plot 
import matplotlib.pyplot as plt 
import seaborn as sns 

# torch 
import torch
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

# nltk 
import re
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


<a id="section-3"></a>
# <span>3. Choix des paramètres</span>
<hr style="border-bottom: solid;background-color:light;color:black;">

In [4]:
MAX_LEN = 512
TRAIN_BATCH_SIZE = 10
VALID_BATCH_SIZE = 10
EPOCHS = 1
LEARNING_RATE = 1e-05

LEN_TEXT = 300
OVERLAP = 50

TRANSFORMER_NAME = "cmarkea/distilcamembert-base"

model_path = "/content/drive/MyDrive/HandOnAI_2_NLP/transformer_model.pth"
test_path = '/content/drive/MyDrive/HandOnAI_2_NLP/fake_test.csv'

<a id="section-4"></a>
# <span>4. Lecture des données</span>
<hr style="border-bottom: solid;background-color:light;color:black;">

In [5]:
df_test = pd.read_csv(test_path)
df_test = df_test.drop(['Unnamed: 0', 'target_name'], axis = 1)

<a id="section-5"></a>
# <span>5. Preprocessing</span>
<hr style="border-bottom: solid;background-color:light;color:black;">

## <span>5.1 Nettoyage des données</span>
<hr style="border-bottom: solid;background-color:light;color:black;">

In [6]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
STOPWORDS = set(stopwords.words('french'))

def clean_text(text):
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text

In [7]:
df_test['data'] = df_test['data'].apply(clean_text)

## <span>5.2 Découpage des données</span>
<hr style="border-bottom: solid;background-color:light;color:black;">

In [8]:
def get_split(text1):
    l_total = []
    l_parcial = []
    if len(text1.split())//(LEN_TEXT - OVERLAP) >0:
        n = len(text1.split())//(LEN_TEXT - OVERLAP)
    else: 
        n = 1
    for w in range(n):
        if w == 0:
            l_parcial = text1.split()[:LEN_TEXT]
            l_total.append(" ".join(l_parcial))
        else:
            l_parcial = text1.split()[w*(LEN_TEXT - OVERLAP):w*(LEN_TEXT - OVERLAP) + LEN_TEXT]
            l_total.append(" ".join(l_parcial))
    return l_total

In [9]:
df_test['text_split'] = df_test['data'].apply(get_split)
df_test['len_split'] = df_test['text_split'].apply(lambda x: len(x))

## <span>5.3 Reformulation du labels</span>
<hr style="border-bottom: solid;background-color:light;color:black;">

In [10]:
def create_df(df): 
  train_l = []
  label_l = []
  for idx,row in df.iterrows():
      for l in row['text_split']:
          train_l.append(l)
          label_l.append([1 if row['label'] == i else 0 for i in range(2)])

  return pd.DataFrame({'data':train_l, 'label':label_l})

In [11]:
cleaned_df_test = create_df(df_test)

## <span>5.4 Création du dataset</span>
<hr style="border-bottom: solid;background-color:light;color:black;">

In [12]:
class CustomDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len, is_target = True):
        self.tokenizer = tokenizer
        self.df = dataframe
        self.text = dataframe.data
        self.max_len = max_len
        if is_target: 
          self.targets = self.df.label
        else: 
          self.targets = None

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True,
            truncation=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]

        if self.targets is None: 
          return {
              'ids': torch.tensor(ids, dtype=torch.long),
              'mask': torch.tensor(mask, dtype=torch.long),
              'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long)
          }
        else: 
          return {
              'ids': torch.tensor(ids, dtype=torch.long),
              'mask': torch.tensor(mask, dtype=torch.long),
              'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
              'targets': torch.tensor(self.targets[index], dtype=torch.float)
          }

In [13]:
tokenizer = AutoTokenizer.from_pretrained(TRANSFORMER_NAME)
testing_set = CustomDataset(cleaned_df_test, tokenizer, MAX_LEN)

Downloading:   0%|          | 0.00/236 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/732 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/811k [00:00<?, ?B/s]

## <span>5.5 Création du dataloader</span>
<hr style="border-bottom: solid;background-color:light;color:black;">

In [14]:
test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': False,
                'num_workers': 0
                }

testing_loader = DataLoader(testing_set, **test_params)

<a id="section-6"></a>
# <span>6. Chargement du modèle</span>
<hr style="border-bottom: solid;background-color:light;color:black;">

In [15]:
class BERTClass(torch.nn.Module):
    def __init__(self):
      super(BERTClass, self).__init__()
      self.l1 = CamembertModel.from_pretrained(TRANSFORMER_NAME)
      self.l3 = torch.nn.Linear(768, 2) #2 = binary classification
    
    def forward(self, ids, mask, token_type_ids):
      output_1= self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids)
      output = self.l3(output_1['pooler_output'])

      return output

In [22]:
def load_checkpoint(filepath):
    #checkpoint = torch.load(filepath, map_location=torch.device('cpu'))
    checkpoint = torch.load(filepath)
    model = checkpoint['model']
    model.load_state_dict(checkpoint['state_dict'])
    for parameter in model.parameters():
        parameter.requires_grad = False
    
    model.eval()
    model.to(device)
    return model

In [23]:
model = load_checkpoint(model_path)

<a id="section-7"></a>
# <span>7. Prédiction du modèle avec les données splité</span>
<hr style="border-bottom: solid;background-color:light;color:black;">

In [24]:
def validation():
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
      for _, data in enumerate(testing_loader, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)
        outputs = model(ids, mask, token_type_ids)
        fin_targets.extend(targets.cpu().detach().numpy().tolist())

        m = torch.nn.Softmax(dim=1)
        fin_outputs.extend(torch.round(m(outputs)).cpu().detach().numpy().tolist())

    return fin_outputs, fin_targets

In [25]:
outputs, targets = validation()
accuracy = metrics.accuracy_score(targets, outputs)
f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
f1_score_macro = metrics.f1_score(targets, outputs, average='macro')
print(f"Accuracy Score = {accuracy}")
print(f"F1 Score (Micro) = {f1_score_micro}")
print(f"F1 Score (Macro) = {f1_score_macro}")

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2336: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


KeyboardInterrupt: ignored

<a id="section-8"></a>
# <span>8. Prédiction du modèle avec les données d'origine</span>
<hr style="border-bottom: solid;background-color:light;color:black;">

In [ ]:
cleaned_df_test['pred'] = outputs
pos = 0
df_test['pred'] = [list() for x in range(len(df_test.index))]
for idx,row in df_test.iterrows():
  for i in range(row['len_split']): 
    row['pred'].append(cleaned_df_test.loc[pos]['pred'])
    pos += 1

In [ ]:
df_test['prediction'] = df_test['pred'].apply(lambda x: [1, 0] if np.argmax(np.sum(x, axis = 0)) == 0 else [0, 1])
df_test['label_pred'] = df_test['pred'].apply(lambda x: np.argmax(np.sum(x, axis = 0)))

In [ ]:
accuracy = metrics.accuracy_score(df_test['label_pred'], df_test['label'])
f1_score_micro = metrics.f1_score(df_test['label_pred'], df_test['label'], average='micro')
f1_score_macro = metrics.f1_score(df_test['label_pred'], df_test['label'], average='macro')
print(f"Accuracy Score = {accuracy}")
print(f"F1 Score (Micro) = {f1_score_micro}")
print(f"F1 Score (Macro) = {f1_score_macro}")